# Classificação de Texto com Transformer - DistilBERT

In [ ]:
%pip install evaluate

In [3]:
# Importando bibliotecas
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

In [4]:
# Carregar dataset IMDB
dataset = load_dataset('imdb')
dataset = dataset.shuffle(seed=42)

c:\Users\josaf\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\josaf\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 327970.05 examples/s]


In [6]:
# Tokenização
tokenizer =  DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
def preprocess(examples):
  return tokenizer(examples['text'], truncation=True, padding=True)
encoded = dataset.map(preprocess, batched=True)

Map: 100%|██████████| 50000/50000 [00:19<00:00, 2621.54 examples/s]


In [7]:
# Modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Métrica
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = logits.argmax(-1)
  return accuracy.compute(predictions=preds, references=labels)

In [9]:
# Treinamento
training_args = TrainingArguments(
  output_dir="./resultados",
  evaluation_strategy="epoch",
  save_strategy="epoch",
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  num_train_epochs=2,
  weight_decay=0.01,
  load_best_model_at_end=True,
  logging_dir="./logs",
  logging_steps=10
)

c:\Users\josaf\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=encoded['train'].select(range(500)),
  eval_dataset=encoded['test'].select(range(100)),
  tokenizer=tokenizer,
  compute_metrics=compute_metrics
)

C:\Users\josaf\AppData\Local\Temp\ipykernel_12064\467453701.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# Executa o modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.376000,0.363116,0.850000
2,0.146000,0.372396,0.870000


TrainOutput(global_step=126, training_loss=0.4086626332903665, metrics={'train_runtime': 986.4645, 'train_samples_per_second': 1.014, 'train_steps_per_second': 0.128, 'total_flos': 132467398656000.0, 'train_loss': 0.4086626332903665, 'epoch': 2.0})

In [14]:
# Resultado
trainer.evaluate()

{'eval_loss': 0.36311641335487366,
 'eval_accuracy': 0.85,
 'eval_runtime': 22.0048,
 'eval_samples_per_second': 4.544,
 'eval_steps_per_second': 0.591,
 'epoch': 2.0}